In [23]:
'''
This python program implements the algorithm proposed in section 3 of the article "On using Deep Reinforcement Learning to balance
Power Consumption and latency100_10_0_20 in 5G NR" in which is an optimization algorithm to derive the best C-DRX parameters and Bandwitch Part
(BWP) configuration while guaranteeing a low Power Comsuption (PC) and avoiding the latency100_10_0_20 overflow
'''

import random as rd
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# K UEs
#UEs = []
UEs = {}
UEsAux = []
# UEs = [ue0, ue1]
#ue0 = [[dij], [fij], [gij], [wij]]

#----- C-DRX Parameters -----

T = [10*x for x in range(0, 40)]


# available bandwidth
W = 2500000  #2.5 GHz

# Subcarrier spacing
sp = 692.5  #692.5 KHz

# Frequency of one PRB
fp = 180  #180 KHz

# Total number of PRBs
prbs = int((W - (2*sp))//fp)


# URLLC DRX possible configurations

deltaU = [2, 3, 4, 5, 6, 7, 8, 10, 14, 16, 20, 30, 32, 35, 40, 64, 80, 128, 160,
          256, 320, 512, 640]  # Set of Cycle Lenghts
fiU = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]  # Set of ON periods

dictU = {key: [] for key in deltaU}
for deltai in dictU.keys():
  for onL in fiU:
    if ((onL + int(0.5*onL)) <= deltai) and (deltai - onL + int(0.5*onL)) <= 5:
      dictU[deltai].append(onL)

aux = []
for key, value in dictU.items():
  if len(value) == 0:
    aux.append(key)

for key in aux:
  dictU.pop(key)


omegaU = [25000, 30000, 40000, 50000]  #MHz

# eMBB DRX possible configurations
deltaE = [10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
          2048, 2560, 5120, 10240]
fiE = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]
gamaE = [0, 10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
         2048, 2560, 5120, 10240]


dictE = {key:{} for key in deltaE}
for deltai in dictE.keys():
  for onL in fiE:
    if onL <= deltai:
      dictE[deltai][onL] = []
      for offL in gamaE:
        if (offL + onL) <= deltai:
          dictE[deltai][onL].append(offL)

omegaE = [5000, 10000, 15000, 20000, 25000, 30000]  #MHz


# Posssible Cell_Sites
c = [1, 2]

# Power Comsuption
Pit = []
Pmax = 80


# Delay constraint URLLC
Limax = 4  #4ms
Lit = []

beta = []

In [24]:
def adiciona(estatico, x):
  if estatico == 1:  # pior caso URLLC (Cycle Length = 20ms; onTime = 1ms; offset = 2ms)
    dij = list(x[0:len(dictU.keys())])
    if dij.count(1) == 0:
      randindex = rd.randint(0, len(dictU.keys()) - 1)
      dij[randindex] = 1

    deltai = deltaU[dij.index(1)]

    onsP = dictU[deltai]
    fij = list(x[23: len(onsP) + 23])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    wij = list(x[47:51])
    cij = list(x[51:53])
    soma = 0

    UEsAux.append({"Usuário": "URLLC", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": int(0.5*onA), "BWP [MHz]": omegaU[wij.index(1)], "Packet Size [bytes]": 360,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 50
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

  elif estatico == 2:
    dij = list(x[53:73])
    deltai = deltaE[dij.index(1)]
    dictdi = dictE[deltai]
    onsP = list(dictdi.keys())
    fij = list(x[73: len(onsP) + 73])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    offsP = dictdi[onA]

    gij = list(x[103: len(offsP) + 103])
    if gij.count(1) == 0:
      randindex = rd.randint(0, len(offsP) - 1)
      gij[randindex] = 1

    offA = offsP[gij.index(1)]

    wij = list(x[123:129])
    cij = list(x[129:131])
    soma = 0

    UEsAux.append({"Usuário": "eMBB", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": offA, "BWP [MHz]": omegaE[wij.index(1)], "Packet Size [bytes]": 6516,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 20
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

In [25]:
def inicializaUE():
  keys = UEsAux[0].keys()
  UEs = {key:[] for key in keys}


  for ue in UEsAux:
    for keys2, values in ue.items():
      UEs[keys2].append(values)

  df = pd.DataFrame(UEs)
  return df

In [26]:
def offLs(df):
  #offLs = np.array(df['offset [ms]'])
  #return offLs
  pass

In [27]:
def offsets_(cycles):
  offsetsTimes = []

  for cycle in cycles:
    qtdCycle = 400/cycle
    aux = []
    i = 0
    while i <= qtdCycle:
      aux.append(cycle * i)
      i += 1

    offsetsTimes.append(aux)

  return offsetsTimes

In [28]:
def onTimes_(offs, cycles):
  onTimes = []
  for offL, cycle in zip(offs, cycles):
    qtdCycle = 400/cycle
    aux = [offL]
    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle * i))
      i += 1

    onTimes.append(aux)

  return onTimes

In [29]:
def inTimes_(cycles, onLs, offs):
  in_ = []
  for cycle, onL, offL in zip(cycles, onLs, offs):
    qtdCycle = 400/cycle

    aux = [(offL + onL + 1)]

    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle* i))
      i += 1

    in_.append(aux)

  return in_

Para cada usuário, haverá a lista de ins, ons e offs, para poder calcular as latências

In [30]:
def scheduler(df):
  df = df.sort_values(by = 'Cell_Site', ascending = True)
  df1 = df.query("Cell_Site == 1")
  df2 = df.query("Cell_Site == 2")
  snirUE1 = np.array([])
  snirUE2 = np.array([])
  k = 0
  prbs = 0
  for row1 in df1.itertuples():
    bwp1 = row1[5]
    if bwp1 == 5000:
      prbs = 25
      k = 1
    elif bwp1 == 10000:
      prbs = 52
      k = 2
    elif bwp1 == 15000:
      prbs = 79
      k = 3
    elif bwp1 == 20000:
      prbs = 106
      k = 4
    elif bwp1 == 25000:
      prbs = 133
      k = 5
    elif bwp1 == 30000:
      prbs = 160
      k = 6
    elif bwp1 == 40000:
      prbs = 216
      k = 8
    elif bwp1 == 50000:
      prbs = 270
      k = 10

    gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks1 = np.array(rd.choices(range(10, 30), weights=range(10, 30), k = (prbs//k)))
    signal1 = np.sum(gains1*pBlocks1)

    sliceType1 = row1[1]
    interference = np.array([])
    for row2 in df2.itertuples():
      sliceType2 = row2[1]
      bwp2 = row2[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal2 = np.sum(gains2*pBlocks2)

        interference = np.append(interference, signal2)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal1/(totalInterferenceUe + 114)
    snirUE1 = np.append(snirUE1, snirUE)

  for row2 in df2.itertuples():
    bwp2 = row2[5]
    if bwp2 == 5000:
      prbs = 25
      k = 1
    elif bwp2 == 10000:
      prbs = 52
      k = 2
    elif bwp2 == 15000:
      prbs = 79
      k = 3
    elif bwp2 == 20000:
      prbs = 106
      k = 4
    elif bwp2 == 25000:
      prbs = 133
      k = 5
    elif bwp2 == 30000:
      prbs = 160
      k = 6
    elif bwp2 == 40000:
      prbs = 216
      k = 8
    elif bwp2 == 50000:
      prbs = 270
      k = 10

    gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
    signal2 = np.sum(gains2*pBlocks2)

    sliceType2 = row2[1]
    interference = np.array([])
    for row1 in df1.itertuples():
      sliceType1 = row1[1]
      bwp1 = row1[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks1 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal1 = np.sum(gains1*pBlocks1)

        interference = np.append(interference, signal1)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal2/(totalInterferenceUe + 114)
    snirUE2 = np.append(snirUE2, snirUE)

  SNIR = np.concatenate((snirUE1, snirUE2))
  SNIRlist = SNIR.tolist()
  df.loc[:, 'SNIR'] = SNIRlist
  return df


In [31]:
import math
def shannonCapacity(df):
  snirs = list(df['SNIR'])
  bwps = list(df['BWP [MHz]'])
  shannonCapacity = []

  for bwp, snirUE in zip(bwps, snirs):
    capacity = bwp*math.log2(1 + snirUE)

    shannonCapacity.append(capacity)

  df['Shannon Capacity'] = shannonCapacity
  #print(df)
  return df


In [32]:
def timeOff(offs, ons, tcs):
  taO = []
  aux = []

  for tc2 in tcs:
    for off, on in zip(offs, ons):
      if offs[0] == ons[0]:
        aux.append(0)

      elif tc2 in range(off, on):
        aux.append(on - tc2)
      else:
        aux.append(0)

    taO.append(sum(aux))
    aux.clear()

  return np.array(taO)

In [33]:
def timeIn(offs, ons, ins, tcs, onL, cycle):
  aux = []
  taI = []
  for tc2 in tcs:
    for inn, off, on in zip(ins, offs, ons):
      if (tc2 in range(inn, (off + cycle + 1))):
        if cycle == onL:
          aux.append(0)
        else:
          aux.append((on + cycle) - tc2)
      else:
        aux.append(0)

    taI.append(sum(aux))
    aux.clear()

  return np.array(taI)

In [34]:
from scipy.stats import poisson

In [35]:
def arrivalTimePoisson():
  mean_rate = 1//5
  qtds = [y for y in range(1, 401)]
  probability = poisson.pmf(qtds, mean_rate)
  aux = {}

  for qtd, prob in zip(qtds, probability):
    aux[qtd] = prob

  values = []
  values.extend(aux.values())
  maxProb = max(values)
  for chave in aux.keys():
    if aux[chave] == maxProb:
      qtdMax = chave

  period = 400//qtdMax
  tc = [timeA for timeA in range(0, 401, period)]
  return tc

In [36]:
def latencyCycle(df, tc):
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onLs = np.array(df['onTime [ms]'])
  inTimes = inTimes_(cycles, onLs, offLs)
  offsets = offsets_(cycles)
  onTimes = onTimes_(offLs, cycles)

  Lc = []
  for on, off, in_ in zip(onTimes, offsets, inTimes):
    onLUE = onLs[onTimes.index(on)]
    cycleUE = cycles[onTimes.index(on)]

    taI = timeIn(off, on, in_, tc, onLUE, cycleUE)
    taO = timeOff(off, on, tc)
    latencyCycle = taI + taO
    Lc.append(latencyCycle.tolist())

  df['Latency_Cycle'] = Lc
  #print(df)
  return df

In [37]:
from itertools import cycle
def DCI(df, tcs):
  kmax = []

  bwps = np.array(df['BWP [MHz]'])

  onL = np.array(df['onTime [ms]'])

  cycle = np.array(df['Cycle Length [ms]'])

  for bwp in bwps:
    if bwp == 25000:
      kmax.append(5)
    elif bwp == 30000:
      kmax.append(6)
    elif bwp == 40000:
      kmax.append(8)
    elif bwp == 20000:
      kmax.append(4)
    elif bwp == 5000:
      kmax.append(1)
    elif bwp == 10000:
      kmax.append(2)
    elif bwp == 15000:
      kmax.append(3)
    elif bwp == 50000:
      kmax.append(10)


  auxx = [0]*len(tcs)
  aux1 = [auxx]*len(kmax)
  for tc in range(len(tcs)):
    auxList = []
    for k in kmax:
      if kmax.index(k) not in auxList:
        indexes = [kmax.index(k)]
        for k2 in kmax[kmax.index(k):]:
          if k2 == k:
            indexes.append(kmax.index(k2))

      auxList.extend(indexes)
      qtdGroup = onL[indexes[0]]//k
      for ue in indexes:
        group = 0
        kaux = 0
        if aux1[ue][tc] == 0:
          aux1[ue][tc] = 0
          sample = rd.sample(indexes, k)
          while (ue not in sample):
            group += 1
            aux1[ue][tc] += 1
            for ue2 in sample:
              if group > qtdGroup and group % qtdGroup == 0:
                kaux = 0

              if group < qtdGroup:
                if kaux < k:
                  aux1[ue2][tc] += 1
                elif kaux >= k and kaux < 2*k:
                  aux1[ue2][tc] += 2
                elif kaux >= 2*k and kaux < 3*k:
                  aux1[ue2][tc] += 3
                elif kaux >= 3*k and kaux < 4*k:
                  aux1[ue2][tc] += 4
                elif kaux >= 4*k and kaux < 5*k:
                  aux1[ue2][tc] += 5
                elif kaux >= 5*k and kaux < 6*k:
                  aux1[ue2][tc] += 6

                kaux += 1
              elif group >= qtdGroup and group < 2* qtdGroup:
                if kaux < k:
                  aux1[ue2][tc] = cycle[ue2]
                elif kaux >= k and kaux < 2*k:
                  aux1[ue2][tc] = cycle[ue2] + 1
                elif kaux >= 2*k and kaux < 3*k:
                  aux1[ue2][tc] = cycle[ue2] + 2
                elif kaux >= 3*k and kaux < 4*k:
                  aux1[ue2][tc] = cycle[ue2] + 3
                elif kaux >= 4*k and kaux < 5*k:
                  aux1[ue2][tc] = cycle[ue2] + 4
                elif kaux >= 5*k and kaux < 6*k:
                  aux1[ue2][tc] = cycle[ue2] + 5
                elif kaux >= 6*k and kaux < 7*k:
                  aux1[ue2][tc] = cycle[ue2] + 6

                kaux += 1
              elif group >= 2*qtdGroup and group < 3*qtdGroup:
                if kaux < k:
                  aux1[ue2][tc] = 2*cycle[ue2]
                elif kaux >= k and kaux < 2*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 1
                elif kaux >= 2*k and kaux < 3*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 2
                elif kaux >= 3*k and kaux < 4*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 3
                elif kaux >= 4*k and kaux < 5*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 4
                elif kaux >= 5*k and kaux < 6*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 5
                elif kaux >= 6*k and kaux < 7*k:
                  aux1[ue2][tc] = 2*cycle[ue2] + 6

                kaux += 1
              elif group >= 3*qtdGroup:
                if kaux < k:
                  aux1[ue2][tc] = 3*cycle[ue2]
                elif kaux >= k and kaux < 2*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 1
                elif kaux >= 2*k and kaux < 3*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 2
                elif kaux >= 3*k and kaux < 4*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 3
                elif kaux >= 4*k and kaux < 5*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 4
                elif kaux >= 5*k and kaux < 6*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 5
                elif kaux >= 6*k and kaux < 7*k:
                  aux1[ue2][tc] = 3*cycle[ue2] + 6

                kaux += 1

            sample = rd.sample(indexes, k)


  df['DCI_time'] = aux1
  return df

In [38]:
def transmissionTime(df, tcs):
  sizes = np.array(df['Packet Size [bytes]'])
  capacities = np.array(df['Shannon Capacity'])
  tT = []

  for size, capacity in zip(sizes, capacities):
    aux = []
    for tc in tcs:
      aux.append(size/capacity)  #196.08 bytes/ms = 196608 bytes/s = 1.5mbs
    tT.append(aux)

  df['Transmission_time'] = tT

  return df

In [39]:
def totalLatency(df):
  latencyaux = list(df['Latency_Cycle'])
  tTaux = list(df['Transmission_time'])
  dciaux = list(df['DCI_time'])

  latencyCycle = np.array(latencyaux)
  dci = np.array(dciaux)
  tT = np.array(tTaux)

  totalLatencyaux = latencyCycle + dci
  totalLatency = totalLatencyaux + tT
  df['Total_Latency'] = totalLatency.tolist()
  #print(df)
  return df

In [40]:
def power(df):
  p = []
  onLs = np.array(df['onTime [ms]'])
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onTimes = onTimes_(offLs, cycles)
  for ons in onTimes:
    onLUE = onLs[onTimes.index(ons)]
    for oni in ons:
      for i in range(onLUE):
        p.append(1)

    Pit.append(sum(p))

In [41]:
def lista_simples(lista):
  if isinstance(lista, list):
      return [sub_elem for elem in lista for sub_elem in lista_simples(elem)]
  else:
      return [lista]

In [42]:
def objectiveFunction(x, nURLLC, nEMBB):
  # x[0] = variável para otimizar a latência em função do omega
  # x[1] = variável para calcular a quantidade de ciclo ON

  #qtdCy = 400
  #tcx = int(x[0])
  #tc = [tcx for tcx in range (tcx, qtdCy, tcx)]
  tc = arrivalTimePoisson()

  UEs.clear()
  UEsAux.clear()
  if nURLLC > nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nEMBB:
        adiciona(2, x)
      else:
        adiciona(1, x)
  elif nURLLC < nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nURLLC:
        adiciona(1, x)
      else:
        adiciona(2, x)

  df = inicializaUE()
  df = scheduler(df)
  df = shannonCapacity(df)
  df = latencyCycle(df, tc)
  df = DCI(df, tc)
  df = transmissionTime(df, tc)
  df = totalLatency(df)
  #print(df)
  latency = (np.array(df['Total_Latency']))
  auxlist = []
  auxlist.append(latency.tolist())
  Lit.extend(lista_simples(auxlist))
  power(df)


  sum2 = 0


  for i in range(0, len(UEs)):
      sum1 = 0
      sum2 = 0
      for j in range(0, len(T)):
          aux = (beta[i] * Pit[i])/ Pmax
          sum1 += aux

      sum2 += sum1
  return sum2


#restricoes
def restricao(x):
  sum1 = sum(x[0:23])
  sum2 = sum(x[23:47])
  sum3 = sum(x[47:51])
  sum4 = sum(x[51:53])
  sum5 = sum(x[53:73])
  sum6 = sum(x[73:123])
  sum7 = sum(x[123:129])
  sum8 = sum(x[129:131])
  return (sum1 + sum2 + sum3 + sum4 + sum5 + sum6 + sum7 + sum8 - 8)

cons = [{'type': 'ineq', 'fun': restricao}]


# limites
#a = [10, 20, 50, 100, 200]
#a = [5, 10, 20, 50, 100]
#b = [1000, 100000]  # Valores possíveis de wij

a = tuple(131*[[0, 1]])


bnds = a



In [43]:
from time import time

In [44]:
# estimativa inicial
x0 = 131*[1]

#Vetores para guardar a latência em cada caso
latency_20_80_1 = []
latency_40_60_1 = []
latency_60_40_1 = []
latency_80_20_1 = []

#Vetores para guardar a quantidade de usuários na rede

#Vetores para guardar a potência em cada caso
power_20_80_1 = []
power_40_60_1 = []
power_60_40_1 = []
power_80_20_1 = []

NURLLC = 20
NeMBB = 80

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_20_80_1.extend(Lit)
  power_20_80_1.extend(Pit)

Lit.clear()
Pit.clear()
UEs.clear()

In [45]:
NURLLC = 40
NeMBB = 60

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_40_60_1.extend(Lit)
  power_40_60_1.extend(Pit)


Lit.clear()
Pit.clear()
UEs.clear()

In [46]:
NURLLC = 60
NeMBB = 40

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_60_40_1.extend(Lit)
  power_60_40_1.extend(Pit)


Lit.clear()
Pit.clear()
UEs.clear()

In [ ]:
NURLLC = 80
NeMBB = 20

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_80_20_1.extend(Lit)
  power_80_20_1.extend(Pit)


Lit.clear()
Pit.clear()
UEs.clear()

In [ ]:
sortL_1 = np.sort(latency_20_80_1)
sortL2_1 = np.sort(latency_40_60_1)
sortL3_1 =  np.sort(latency_60_40_1)
sortL4_1 = np.sort(latency_80_20_1)

pL_1 = 1.0 * np.arange(len(sortL_1)) / float(len(sortL_1) - 1)
pL2_1 = 1.0 * np.arange(len(sortL2_1)) / float(len(sortL2_1) - 1)
pL3_1 = 1.0 * np.arange(len(sortL3_1)) / float(len(sortL3_1) - 1)
pL4_1 = 1.0 * np.arange(len(sortL4_1)) / float(len(sortL4_1) - 1)

plt.plot(sortL_1, pL_1, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_1, pL2_1, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_1, pL3_1, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_1, pL4_1, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
sortP_1 = np.sort(power_20_80_1)
sortP2_1 = np.sort(power_40_60_1)
sortP3_1 =  np.sort(power_60_40_1)
sortP4_1 = np.sort(power_80_20_1)

pP_1 = 1.0 * np.arange(len(sortL_1)) / float(len(sortL_1) - 1)
pP2_1 = 1.0 * np.arange(len(sortL2_1)) / float(len(sortL2_1) - 1)
pP3_1 = 1.0 * np.arange(len(sortL3_1)) / float(len(sortL3_1) - 1)
pP4_1 = 1.0 * np.arange(len(sortL4_1)) / float(len(sortL4_1) - 1)

plt.plot(sortP_1, pP_1, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_1, pP2_1, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_1, pP3_1, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_1, pP4_1, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
'''
This python program implements the algorithm proposed in section 3 of the article "On using Deep Reinforcement Learning to balance
Power Consumption and latency100_10_0_20 in 5G NR" in which is an optimization algorithm to derive the best C-DRX parameters and Bandwitch Part
(BWP) configuration while guaranteeing a low Power Comsuption (PC) and avoiding the latency100_10_0_20 overflow
'''

import random as rd
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# K UEs
#UEs = []
UEs = {}
UEsAux = []
# UEs = [ue0, ue1]
#ue0 = [[dij], [fij], [gij], [wij]]

#----- C-DRX Parameters -----

T = [10*x for x in range(0, 40)]


# available bandwidth
W = 2500000  #2.5 GHz

# Subcarrier spacing
sp = 692.5  #692.5 KHz

# Frequency of one PRB
fp = 180  #180 KHz

# Total number of PRBs
prbs = int((W - (2*sp))//fp)


# URLLC DRX possible configurations

deltaU = [2, 3, 4, 5, 6, 7, 8, 10, 14, 16, 20, 30, 32, 35, 40, 64, 80, 128, 160,
          256, 320, 512, 640]  # Set of Cycle Lenghts
fiU = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]  # Set of ON periods

dictU = {key: [] for key in deltaU}
for deltai in dictU.keys():
  for onL in fiU:
    if ((onL + int(0.5*onL)) <= deltai) and (deltai - onL + int(0.5*onL)) <= 5:
      dictU[deltai].append(onL)

aux = []
for key, value in dictU.items():
  if len(value) == 0:
    aux.append(key)

for key in aux:
  dictU.pop(key)


omegaU = [25000, 30000, 40000, 50000]  #MHz

# eMBB DRX possible configurations
deltaE = [10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
          2048, 2560, 5120, 10240]
fiE = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]
gamaE = [0, 10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
         2048, 2560, 5120, 10240]


dictE = {key:{} for key in deltaE}
for deltai in dictE.keys():
  for onL in fiE:
    if onL <= deltai:
      dictE[deltai][onL] = []
      for offL in gamaE:
        if (offL + onL) <= deltai:
          dictE[deltai][onL].append(offL)

omegaE = [5000, 10000, 15000, 20000, 25000, 30000]  #MHz


# Posssible Cell_Sites
c = [1, 2]

# Power Comsuption
Pit = []
Pmax = 80


# Delay constraint URLLC
Limax = 4  #4ms
Lit = []

beta = []

In [ ]:
def adiciona(estatico, x):
  if estatico == 1:  # pior caso URLLC (Cycle Length = 20ms; onTime = 1ms; offset = 2ms)
    dij = list(x[0:len(dictU.keys())])
    if dij.count(1) == 0:
      randindex = rd.randint(0, len(dictU.keys()) - 1)
      dij[randindex] = 1

    deltai = deltaU[dij.index(1)]

    onsP = dictU[deltai]
    fij = list(x[23: len(onsP) + 23])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    wij = list(x[47:51])
    cij = list(x[51:53])
    soma = 0

    UEsAux.append({"Usuário": "URLLC", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": int(0.5*onA), "BWP [MHz]": omegaU[wij.index(1)], "Packet Size [bytes]": 360,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 50
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

  elif estatico == 2:
    dij = list(x[53:73])
    deltai = deltaE[dij.index(1)]
    dictdi = dictE[deltai]
    onsP = list(dictdi.keys())
    fij = list(x[73: len(onsP) + 73])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    offsP = dictdi[onA]

    gij = list(x[103: len(offsP) + 103])
    if gij.count(1) == 0:
      randindex = rd.randint(0, len(offsP) - 1)
      gij[randindex] = 1

    offA = offsP[gij.index(1)]

    wij = list(x[123:129])
    cij = list(x[129:131])
    soma = 0

    UEsAux.append({"Usuário": "eMBB", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": offA, "BWP [MHz]": omegaE[wij.index(1)], "Packet Size [bytes]": 6516,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 20
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

In [ ]:
def inicializaUE():
  keys = UEsAux[0].keys()
  UEs = {key:[] for key in keys}


  for ue in UEsAux:
    for keys2, values in ue.items():
      UEs[keys2].append(values)

  df = pd.DataFrame(UEs)
  return df

In [ ]:
def offLs(df):
  #offLs = np.array(df['offset [ms]'])
  #return offLs
  pass

In [ ]:
def offsets_(cycles):
  offsetsTimes = []

  for cycle in cycles:
    qtdCycle = 400/cycle
    aux = []
    i = 0
    while i <= qtdCycle:
      aux.append(cycle * i)
      i += 1

    offsetsTimes.append(aux)

  return offsetsTimes

In [ ]:
def onTimes_(offs, cycles):
  onTimes = []
  for offL, cycle in zip(offs, cycles):
    qtdCycle = 400/cycle
    aux = [offL]
    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle * i))
      i += 1

    onTimes.append(aux)

  return onTimes

In [ ]:
def inTimes_(cycles, onLs, offs):
  in_ = []
  for cycle, onL, offL in zip(cycles, onLs, offs):
    qtdCycle = 400/cycle

    aux = [(offL + onL + 1)]

    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle* i))
      i += 1

    in_.append(aux)

  return in_

Para cada usuário, haverá a lista de ins, ons e offs, para poder calcular as latências

In [ ]:
def scheduler(df):
  df = df.sort_values(by = 'Cell_Site', ascending = True)
  df1 = df.query("Cell_Site == 1")
  df2 = df.query("Cell_Site == 2")
  snirUE1 = np.array([])
  snirUE2 = np.array([])
  k = 0
  prbs = 0
  for row1 in df1.itertuples():
    bwp1 = row1[5]
    if bwp1 == 5000:
      prbs = 25
      k = 1
    elif bwp1 == 10000:
      prbs = 52
      k = 2
    elif bwp1 == 15000:
      prbs = 79
      k = 3
    elif bwp1 == 20000:
      prbs = 106
      k = 4
    elif bwp1 == 25000:
      prbs = 133
      k = 5
    elif bwp1 == 30000:
      prbs = 160
      k = 6
    elif bwp1 == 40000:
      prbs = 216
      k = 8
    elif bwp1 == 50000:
      prbs = 270
      k = 10

    gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks1 = np.array(rd.choices(range(10, 30), weights=range(10, 30), k = (prbs//k)))
    signal1 = np.sum(gains1*pBlocks1)

    sliceType1 = row1[1]
    interference = np.array([])
    for row2 in df2.itertuples():
      sliceType2 = row2[1]
      bwp2 = row2[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal2 = np.sum(gains2*pBlocks2)

        interference = np.append(interference, signal2)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal1/(totalInterferenceUe + 114)
    snirUE1 = np.append(snirUE1, snirUE)

  for row2 in df2.itertuples():
    bwp2 = row2[5]
    if bwp2 == 5000:
      prbs = 25
      k = 1
    elif bwp2 == 10000:
      prbs = 52
      k = 2
    elif bwp2 == 15000:
      prbs = 79
      k = 3
    elif bwp2 == 20000:
      prbs = 106
      k = 4
    elif bwp2 == 25000:
      prbs = 133
      k = 5
    elif bwp2 == 30000:
      prbs = 160
      k = 6
    elif bwp2 == 40000:
      prbs = 216
      k = 8
    elif bwp2 == 50000:
      prbs = 270
      k = 10

    gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
    signal2 = np.sum(gains2*pBlocks2)

    sliceType2 = row2[1]
    interference = np.array([])
    for row1 in df1.itertuples():
      sliceType1 = row1[1]
      bwp1 = row1[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks1 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal1 = np.sum(gains1*pBlocks1)

        interference = np.append(interference, signal1)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal2/(totalInterferenceUe + 114)
    snirUE2 = np.append(snirUE2, snirUE)

  SNIR = np.concatenate((snirUE1, snirUE2))
  SNIRlist = SNIR.tolist()
  df.loc[:, 'SNIR'] = SNIRlist
  return df


In [ ]:
import math
def shannonCapacity(df):
  snirs = list(df['SNIR'])
  bwps = list(df['BWP [MHz]'])
  shannonCapacity = []

  for bwp, snirUE in zip(bwps, snirs):
    capacity = bwp*math.log2(1 + snirUE)

    shannonCapacity.append(capacity)

  df['Shannon Capacity'] = shannonCapacity
  #print(df)
  return df


In [ ]:
def timeOff(offs, ons, tcs):
  taO = []
  aux = []

  for tc2 in tcs:
    for off, on in zip(offs, ons):
      if offs[0] == ons[0]:
        aux.append(0)

      elif tc2 in range(off, on):
        aux.append(on - tc2)
      else:
        aux.append(0)

    taO.append(sum(aux))
    aux.clear()

  return np.array(taO)

In [ ]:
def timeIn(offs, ons, ins, tcs, onL, cycle):
  aux = []
  taI = []
  for tc2 in tcs:
    for inn, off, on in zip(ins, offs, ons):
      if (tc2 in range(inn, (off + cycle + 1))):
        if cycle == onL:
          aux.append(0)
        else:
          aux.append((on + cycle) - tc2)
      else:
        aux.append(0)

    taI.append(sum(aux))
    aux.clear()

  return np.array(taI)

In [ ]:
from scipy.stats import poisson

In [ ]:
def arrivalTimePoisson():
  mean_rate = 1//5
  qtds = [y for y in range(1, 401)]
  probability = poisson.pmf(qtds, mean_rate)
  aux = {}

  for qtd, prob in zip(qtds, probability):
    aux[qtd] = prob

  values = []
  values.extend(aux.values())
  maxProb = max(values)
  for chave in aux.keys():
    if aux[chave] == maxProb:
      qtdMax = chave

  period = 400//qtdMax
  tc = [timeA for timeA in range(0, 401, period)]
  return tc

In [ ]:
def power(df):
  p = []
  onLs = np.array(df['onTime [ms]'])
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onTimes = onTimes_(offLs, cycles)
  for ons in onTimes:
    onLUE = onLs[onTimes.index(ons)]
    for oni in ons:
      for i in range(onLUE):
        p.append(1)

    Pit.append(sum(p))

In [ ]:
def latencyCycle(df, tc):
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onLs = np.array(df['onTime [ms]'])
  inTimes = inTimes_(cycles, onLs, offLs)
  offsets = offsets_(cycles)
  onTimes = onTimes_(offLs, cycles)

  Lc = []
  for on, off, in_ in zip(onTimes, offsets, inTimes):
    onLUE = onLs[onTimes.index(on)]
    cycleUE = cycles[onTimes.index(on)]

    taI = timeIn(off, on, in_, tc, onLUE, cycleUE)
    taO = timeOff(off, on, tc)
    latencyCycle = taI + taO
    Lc.append(latencyCycle.tolist())

  df['Latency_Cycle'] = Lc
  #print(df)
  return df

In [ ]:
from itertools import cycle
def DCI(df):
  df1 = df.query("Usuário == 'URLLC'")
  df1 = df1.sort_values(by = 'Latency_Cycle', ascending=False)

  df2 = df.query("Usuário == 'eMBB'")
  df2 = df2.sort_values(by = 'Latency_Cycle', ascending=False)

  kmax1 = []
  kmax2 = []

  auxlatencysC1 = list(df1['Latency_Cycle'])
  auxlatencysC2 = list(df2['Latency_Cycle'])

  latencysC1 = np.array(auxlatencysC1)
  latencysC2 = np.array(auxlatencysC2)

  #print("Latency c1: ", latencysC1)
  #print("Latency c2: ", latencysC2)
  #print("Shape: ", df.shape)

  rows1, columns1 = latencysC1.shape
  rows2, columns2 = latencysC2.shape

  bwps1 = np.array(df1['BWP [MHz]'])
  bwps2 = np.array(df2['BWP [MHz]'])

  onL1 = np.array(df1['onTime [ms]'])
  onL2 = np.array(df2['onTime [ms]'])

  cycle1 = np.array(df1['Cycle Length [ms]'])
  cycle2 = np.array(df2['Cycle Length [ms]'])


  for bwp in bwps1:
    if bwp == 25000:
      kmax1.append(5)
    elif bwp == 30000:
      kmax1.append(6)
    elif bwp == 40000:
      kmax1.append(8)
    elif bwp == 50000:
      kmax1.append(10)

  for bwp in bwps2:
    if bwp == 25000:
      kmax2.append(5)
    elif bwp == 30000:
      kmax2.append(6)
    elif bwp == 20000:
      kmax2.append(4)
    elif bwp == 5000:
      kmax2.append(1)
    elif bwp == 10000:
      kmax2.append(2)
    elif bwp == 15000:
      kmax2.append(3)

  auxx = [0]*columns1
  aux1 = [auxx]*rows1
  for column1 in range(columns1):
    auxList = []
    for kmax in kmax1:
      if kmax1.index(kmax) not in auxList:
        indexes = [kmax1.index(kmax)]
        for kmaxx in kmax1[kmax1.index(kmax):]:
          if kmaxx == kmax:
            indexes.append(kmax1.index(kmaxx))

        auxList.extend(indexes)
        k = 0
        qtdGroup = onL1[indexes[0]]//kmax
        group = 0
        for ue in indexes:
          if k % kmax == 0 and k != 0:
            group += 1
            if group >= qtdGroup and group % qtdGroup == 0:
              k = 0

          if group < qtdGroup:
            if k <= kmax:
              aux1[ue][column1] = 0
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = 7
            elif k > 8*kmax:
              aux1[ue][column1] = 8

            k += 1

          elif group >= qtdGroup and group < qtdGroup*2:
            if k <= kmax:
              aux1[ue][column1] = cycle1[ue]
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = cycle1[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = cycle1[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = cycle1[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = cycle1[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = cycle1[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = cycle1[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = cycle1[ue] + 7
            elif k > 8*kmax:
              aux1[ue][column1] = cycle1[ue] + 8

            k += 1
          elif group >= qtdGroup*2 and group < qtdGroup*3:
            if k <= kmax:
              aux1[ue][column1] = 2*cycle1[ue]
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 7
            elif k > 8*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 8

            k += 1

  auxxx = [0]*columns2
  aux2 = [auxxx]*rows2
  for column2 in range(columns2):
    auxList = []
    for kmax in kmax2:
      if kmax2.index(kmax) not in auxList:
        indexes = [kmax2.index(kmax)]
        for kmaxx in kmax2[kmax2.index(kmax):]:
          if kmaxx == kmax:
            indexes.append(kmax2.index(kmaxx))

        auxList.extend
        k = 0
        qtdGroup = onL2[indexes[0]]//kmax
        group = 0
        for ue in indexes:
          if k % kmax == 0 and k != 0:
            group += 1
            if group >= qtdGroup and group % qtdGroup == 0:
              k = 0

          if group < qtdGroup:
            if k <= kmax:
              aux2[ue][column2] = 0
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = 7
            elif k > 8*kmax:
              aux2[ue][column2] = 8

            k += 1

          elif group >= qtdGroup and group < qtdGroup*2:
            if k <= kmax:
              aux2[ue][column2] = cycle2[ue]
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = cycle2[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = cycle2[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = cycle2[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = cycle2[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = cycle2[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = cycle2[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = cycle2[ue] + 7
            elif k > 8*kmax:
              aux2[ue][column2] = cycle2[ue] + 8

            k += 1
          elif group >= qtdGroup*2 and group < qtdGroup*3:
            if k <= kmax:
              aux2[ue][column2] = 2*cycle2[ue]
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 7
            elif k > 8*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 8

            k += 1

  df1['DCI_time'] = aux1
  df2['DCI_time'] = aux2

  df = pd.concat([df1, df2], ignore_index=True)
  #print(df)
  return df

In [ ]:
def transmissionTime(df, tcs):
  sizes = np.array(df['Packet Size [bytes]'])
  capacities = np.array(df['Shannon Capacity'])
  tT = []

  for size, capacity in zip(sizes, capacities):
    aux = []
    for tc in tcs:
      aux.append(size/capacity)  #196.08 bytes/ms = 196608 bytes/s = 1.5mbs
    tT.append(aux)

  df['Transmission_time'] = tT

  return df

In [ ]:
def totalLatency(df):
  latencyaux = list(df['Latency_Cycle'])
  tTaux = list(df['Transmission_time'])
  dciaux = list(df['DCI_time'])

  latencyCycle = np.array(latencyaux)
  dci = np.array(dciaux)
  tT = np.array(tTaux)

  totalLatencyaux = latencyCycle + dci
  totalLatency = totalLatencyaux + tT
  df['Total_Latency'] = totalLatency.tolist()
  #print(df)
  return df

In [ ]:
def lista_simples(lista):
  if isinstance(lista, list):
      return [sub_elem for elem in lista for sub_elem in lista_simples(elem)]
  else:
      return [lista]

In [ ]:
def objectiveFunction(x, nURLLC, nEMBB):
  # x[0] = variável para otimizar a latência em função do omega
  # x[1] = variável para calcular a quantidade de ciclo ON

  #qtdCy = 400
  #tcx = int(x[0])
  #tc = [tcx for tcx in range (tcx, qtdCy, tcx)]
  tc = arrivalTimePoisson()

  UEs.clear()
  UEsAux.clear()
  if nURLLC > nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nEMBB:
        adiciona(2, x)
      else:
        adiciona(1, x)
  elif nURLLC < nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nURLLC:
        adiciona(1, x)
      else:
        adiciona(2, x)

  df = inicializaUE()
  df = scheduler(df)
  df = shannonCapacity(df)
  df = latencyCycle(df, tc)
  df = DCI(df)
  df = transmissionTime(df, tc)
  df = totalLatency(df)
  #print(df)
  latency = (np.array(df['Total_Latency']))
  auxlist = []
  auxlist.append(latency.tolist())
  Lit.extend(lista_simples(auxlist))
  power(df)


  sum2 = 0


  for i in range(0, len(UEs)):
      sum1 = 0
      sum2 = 0
      for j in range(0, len(T)):
          aux = (beta[i] * Pit[i])/ Pmax
          sum1 += aux

      sum2 += sum1
  return sum2


#restricoes
def restricao(x):
  sum1 = sum(x[0:23])
  sum2 = sum(x[23:47])
  sum3 = sum(x[47:51])
  sum4 = sum(x[51:53])
  sum5 = sum(x[53:73])
  sum6 = sum(x[73:123])
  sum7 = sum(x[123:129])
  sum8 = sum(x[129:131])
  #sum9 = sum(x[131])
  return (sum1 + sum2 + sum3 + sum4 + sum5 + sum6 + sum7 + sum8 - 8)

cons = [{'type': 'ineq', 'fun': restricao}]


# limites
#a = [10, 20, 50, 100, 200]
#a = [5, 10, 20, 50, 100]
#b = [1000, 100000]  # Valores possíveis de wij

a = tuple(131*[[0, 1]])


bnds = a



In [ ]:
from time import time

In [ ]:
# estimativa inicial
x0 = 131*[1]

#Vetores para guardar a latência em cada caso
latency_20_80_2 = []
latency_40_60_2 = []
latency_60_40_2 = []
latency_80_20_2 = []

#Vetores para guardar a quantidade de usuários na rede

#Vetores para guardar a potência em cada caso
power_20_80_2 = []
power_40_60_2 = []
power_60_40_2 = []
power_80_20_2 = []

NURLLC = 20
NeMBB = 80

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_20_80_2.extend(Lit)
  power_20_80_2.extend(Pit)

Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 40
NeMBB = 60

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_40_60_2.extend(Lit)
  power_40_60_2.extend(Pit)


Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 60
NeMBB = 40

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_60_40_2.extend(Lit)
  power_60_40_2.extend(Pit)


Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 80
NeMBB = 20

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_80_20_2.extend(Lit)
  power_80_20_2.extend(Pit)


Lit.clear()
Pit.clear()
UEs.clear()

In [ ]:
sortL_2 = np.sort(latency_20_80_2)
sortL2_2 = np.sort(latency_40_60_2)
sortL3_2 =  np.sort(latency_60_40_2)
sortL4_2 = np.sort(latency_80_20_2)

pL_2 = 1.0 * np.arange(len(sortL_2)) / float(len(sortL_2) - 1)
pL2_2 = 1.0 * np.arange(len(sortL2_2)) / float(len(sortL2_2) - 1)
pL3_2 = 1.0 * np.arange(len(sortL3_2)) / float(len(sortL3_2) - 1)
pL4_2 = 1.0 * np.arange(len(sortL4_2)) / float(len(sortL4_2) - 1)

plt.plot(sortL_2, pL_2, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_2, pL2_2, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_2, pL3_2, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_2, pL4_2, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
sortP_2 = np.sort(power_20_80_2)
sortP2_2 = np.sort(power_40_60_2)
sortP3_2 =  np.sort(power_60_40_2)
sortP4_2 = np.sort(power_80_20_2)

pP_2 = 1.0 * np.arange(len(sortP_2)) / float(len(sortP_2) - 1)
pP2_2 = 1.0 * np.arange(len(sortP2_2)) / float(len(sortP2_2) - 1)
pP3_2 = 1.0 * np.arange(len(sortP3_2)) / float(len(sortP3_2) - 1)
pP4_2 = 1.0 * np.arange(len(sortP4_2)) / float(len(sortP4_2) - 1)

plt.plot(sortP_2, pP_2, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_2, pP2_2, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_2, pP3_2, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_2, pP4_2, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
'''
This python program implements the algorithm proposed in section 3 of the article "On using Deep Reinforcement Learning to balance
Power Consumption and latency100_10_0_20 in 5G NR" in which is an optimization algorithm to derive the best C-DRX parameters and Bandwitch Part
(BWP) configuration while guaranteeing a low Power Comsuption (PC) and avoiding the latency100_10_0_20 overflow
'''

import random as rd
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# K UEs
#UEs = []
UEs = {}
UEsAux = []
# UEs = [ue0, ue1]
#ue0 = [[dij], [fij], [gij], [wij]]

#----- C-DRX Parameters -----

T = [10*x for x in range(0, 40)]


# available bandwidth
W = 2500000  #2.5 GHz

# Subcarrier spacing
sp = 692.5  #692.5 KHz

# Frequency of one PRB
fp = 180  #180 KHz

# Total number of PRBs
prbs = int((W - (2*sp))//fp)


# URLLC DRX possible configurations

deltaU = [2, 3, 4, 5, 6, 7, 8, 10, 14, 16, 20, 30, 32, 35, 40, 64, 80, 128, 160,
          256, 320, 512, 640]  # Set of Cycle Lenghts
fiU = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]  # Set of ON periods

dictU = {key: [] for key in deltaU}
for deltai in dictU.keys():
  for onL in fiU:
    if ((onL + int(0.5*onL)) <= deltai) and (deltai - onL + int(0.5*onL)) <= 5:
      dictU[deltai].append(onL)

aux = []
for key, value in dictU.items():
  if len(value) == 0:
    aux.append(key)

for key in aux:
  dictU.pop(key)


omegaU = [25000, 30000, 40000, 50000]  #MHz

# eMBB DRX possible configurations
deltaE = [10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
          2048, 2560, 5120, 10240]
fiE = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 80, 100, 200, 300, 400, 500,
       600, 800, 1000, 1200, 1600]
gamaE = [0, 10, 20, 32, 40, 60, 64, 70, 80, 128, 160, 256, 320, 512, 640, 1024, 1280,
         2048, 2560, 5120, 10240]


dictE = {key:{} for key in deltaE}
for deltai in dictE.keys():
  for onL in fiE:
    if onL <= deltai:
      dictE[deltai][onL] = []
      for offL in gamaE:
        if (offL + onL) <= deltai:
          dictE[deltai][onL].append(offL)

omegaE = [5000, 10000, 15000, 20000, 25000, 30000]  #MHz


# Posssible Cell_Sites
c = [1, 2]

# Power Comsuption
Pit = []
Pmax = 80


# Delay constraint URLLC
Limax = 4  #4ms
Lit = []

beta = []

In [ ]:
def adiciona(estatico, x):
  if estatico == 1:  # pior caso URLLC (Cycle Length = 20ms; onTime = 1ms; offset = 2ms)
    dij = list(x[0:len(dictU.keys())])
    if dij.count(1) == 0:
      randindex = rd.randint(0, len(dictU.keys()) - 1)
      dij[randindex] = 1

    deltai = deltaU[dij.index(1)]

    onsP = dictU[deltai]
    fij = list(x[23: len(onsP) + 23])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    wij = list(x[47:51])
    cij = list(x[51:53])
    soma = 0

    UEsAux.append({"Usuário": "URLLC", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": int(0.5*onA), "BWP [MHz]": omegaU[wij.index(1)], "Packet Size [bytes]": 360,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 50
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

  elif estatico == 2:
    dij = list(x[53:73])
    deltai = deltaE[dij.index(1)]
    dictdi = dictE[deltai]
    onsP = list(dictdi.keys())
    fij = list(x[73: len(onsP) + 73])
    if fij.count(1) == 0:
      randindex = rd.randint(0, len(onsP) - 1)
      fij[randindex] = 1

    onA = onsP[fij.index(1)]

    offsP = dictdi[onA]

    gij = list(x[103: len(offsP) + 103])
    if gij.count(1) == 0:
      randindex = rd.randint(0, len(offsP) - 1)
      gij[randindex] = 1

    offA = offsP[gij.index(1)]

    wij = list(x[123:129])
    cij = list(x[129:131])
    soma = 0

    UEsAux.append({"Usuário": "eMBB", "Cycle Length [ms]": deltai, "onTime [ms]": onA,
                   "offset [ms]": offA, "BWP [MHz]": omegaE[wij.index(1)], "Packet Size [bytes]": 6516,
                   "Cell_Site": rd.randint(1, 2)})

    soma += 20
    b = 0.4 + (0.6*((soma - 20)/80))
    beta.append(b)

In [ ]:
def inicializaUE():
  keys = UEsAux[0].keys()
  UEs = {key:[] for key in keys}


  for ue in UEsAux:
    for keys2, values in ue.items():
      UEs[keys2].append(values)

  df = pd.DataFrame(UEs)
  return df

In [ ]:
def offLs(df):
  #offLs = np.array(df['offset [ms]'])
  #return offLs
  pass

In [ ]:
def offsets_(cycles):
  offsetsTimes = []

  for cycle in cycles:
    qtdCycle = 400/cycle
    aux = []
    i = 0
    while i <= qtdCycle:
      aux.append(cycle * i)
      i += 1

    offsetsTimes.append(aux)

  return offsetsTimes

In [ ]:
def onTimes_(offs, cycles):
  onTimes = []
  for offL, cycle in zip(offs, cycles):
    qtdCycle = 400/cycle
    aux = [offL]
    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle * i))
      i += 1

    onTimes.append(aux)

  return onTimes

In [ ]:
def inTimes_(cycles, onLs, offs):
  in_ = []
  for cycle, onL, offL in zip(cycles, onLs, offs):
    qtdCycle = 400/cycle

    aux = [(offL + onL + 1)]

    i = 1
    while i <= qtdCycle:
      aux.append(aux[0] + (cycle* i))
      i += 1

    in_.append(aux)

  return in_

Para cada usuário, haverá a lista de ins, ons e offs, para poder calcular as latências

In [ ]:
def scheduler(df):
  df = df.sort_values(by = 'Cell_Site', ascending = True)
  df1 = df.query("Cell_Site == 1")
  df2 = df.query("Cell_Site == 2")
  snirUE1 = np.array([])
  snirUE2 = np.array([])
  k = 0
  prbs = 0
  for row1 in df1.itertuples():
    bwp1 = row1[5]
    if bwp1 == 5000:
      prbs = 25
      k = 1
    elif bwp1 == 10000:
      prbs = 52
      k = 2
    elif bwp1 == 15000:
      prbs = 79
      k = 3
    elif bwp1 == 20000:
      prbs = 106
      k = 4
    elif bwp1 == 25000:
      prbs = 133
      k = 5
    elif bwp1 == 30000:
      prbs = 160
      k = 6
    elif bwp1 == 40000:
      prbs = 216
      k = 8
    elif bwp1 == 50000:
      prbs = 270
      k = 10

    gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks1 = np.array(rd.choices(range(10, 30), weights=range(10, 30), k = (prbs//k)))
    signal1 = np.sum(gains1*pBlocks1)

    sliceType1 = row1[1]
    interference = np.array([])
    for row2 in df2.itertuples():
      sliceType2 = row2[1]
      bwp2 = row2[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal2 = np.sum(gains2*pBlocks2)

        interference = np.append(interference, signal2)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal1/(totalInterferenceUe + 114)
    snirUE1 = np.append(snirUE1, snirUE)

  for row2 in df2.itertuples():
    bwp2 = row2[5]
    if bwp2 == 5000:
      prbs = 25
      k = 1
    elif bwp2 == 10000:
      prbs = 52
      k = 2
    elif bwp2 == 15000:
      prbs = 79
      k = 3
    elif bwp2 == 20000:
      prbs = 106
      k = 4
    elif bwp2 == 25000:
      prbs = 133
      k = 5
    elif bwp2 == 30000:
      prbs = 160
      k = 6
    elif bwp2 == 40000:
      prbs = 216
      k = 8
    elif bwp2 == 50000:
      prbs = 270
      k = 10

    gains2 = np.random.rayleigh(scale = 1, size = (prbs//k))
    pBlocks2 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
    signal2 = np.sum(gains2*pBlocks2)

    sliceType2 = row2[1]
    interference = np.array([])
    for row1 in df1.itertuples():
      sliceType1 = row1[1]
      bwp1 = row1[5]
      if (sliceType1 == sliceType2) and (bwp1 == bwp2):
        gains1 = np.random.rayleigh(scale = 1, size = (prbs//k))
        pBlocks1 = np.array(rd.choices(range(10, 30), weights = range(10, 30), k = (prbs//k)))
        signal1 = np.sum(gains1*pBlocks1)

        interference = np.append(interference, signal1)

    totalInterferenceUe = np.sum(interference)
    snirUE = signal2/(totalInterferenceUe + 114)
    snirUE2 = np.append(snirUE2, snirUE)

  SNIR = np.concatenate((snirUE1, snirUE2))
  SNIRlist = SNIR.tolist()
  df.loc[:, 'SNIR'] = SNIRlist
  return df


In [ ]:
import math
def shannonCapacity(df):
  snirs = list(df['SNIR'])
  bwps = list(df['BWP [MHz]'])
  shannonCapacity = []

  for bwp, snirUE in zip(bwps, snirs):
    capacity = bwp*math.log2(1 + snirUE)

    shannonCapacity.append(capacity)

  df['Shannon Capacity'] = shannonCapacity
  #print(df)
  return df


In [ ]:
def timeOff(offs, ons, tcs):
  taO = []
  aux = []

  for tc2 in tcs:
    for off, on in zip(offs, ons):
      if offs[0] == ons[0]:
        aux.append(0)

      elif tc2 in range(off, on):
        aux.append(on - tc2)
      else:
        aux.append(0)

    taO.append(sum(aux))
    aux.clear()

  return np.array(taO)

In [ ]:
def timeIn(offs, ons, ins, tcs, onL, cycle):
  aux = []
  taI = []
  for tc2 in tcs:
    for inn, off, on in zip(ins, offs, ons):
      if (tc2 in range(inn, (off + cycle + 1))):
        if cycle == onL:
          aux.append(0)
        else:
          aux.append((on + cycle) - tc2)
      else:
        aux.append(0)

    taI.append(sum(aux))
    aux.clear()

  return np.array(taI)

In [ ]:
from scipy.stats import poisson

In [ ]:
def arrivalTimePoisson():
  mean_rate = 1//5
  qtds = [y for y in range(1, 401)]
  probability = poisson.pmf(qtds, mean_rate)
  aux = {}

  for qtd, prob in zip(qtds, probability):
    aux[qtd] = prob

  values = []
  values.extend(aux.values())
  maxProb = max(values)
  for chave in aux.keys():
    if aux[chave] == maxProb:
      qtdMax = chave

  period = 400//qtdMax
  tc = [timeA for timeA in range(0, 401, period)]
  return tc

In [ ]:
def power(df):
  p = []
  onLs = np.array(df['onTime [ms]'])
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onTimes = onTimes_(offLs, cycles)
  for ons in onTimes:
    onLUE = onLs[onTimes.index(ons)]
    for oni in ons:
      for i in range(onLUE):
        p.append(1)

    Pit.append(sum(p))

In [ ]:
def latencyCycle(df, tc):
  cycles = np.array(df['Cycle Length [ms]'])
  offLs = np.array(df['offset [ms]'])
  onLs = np.array(df['onTime [ms]'])
  inTimes = inTimes_(cycles, onLs, offLs)
  offsets = offsets_(cycles)
  onTimes = onTimes_(offLs, cycles)

  Lc = []
  for on, off, in_ in zip(onTimes, offsets, inTimes):
    onLUE = onLs[onTimes.index(on)]
    cycleUE = cycles[onTimes.index(on)]

    taI = timeIn(off, on, in_, tc, onLUE, cycleUE)
    taO = timeOff(off, on, tc)
    latencyCycle = taI + taO
    Lc.append(latencyCycle.tolist())

  df['Latency_Cycle'] = Lc
  #print(df)
  return df

In [ ]:
from itertools import cycle
def DCI(df):
  df1 = df.query("Usuário == 'URLLC'")
  df1 = df1.sort_values(by = 'Latency_Cycle', ascending=False)

  df2 = df.query("Usuário == 'eMBB'")
  df2 = df2.sort_values(by = 'Latency_Cycle', ascending=False)

  kmax1 = []
  kmax2 = []

  auxlatencysC1 = list(df1['Latency_Cycle'])
  auxlatencysC2 = list(df2['Latency_Cycle'])

  #auxCapacity1 = list(df1['Shannon Capacity'])
  #auxCapacity2 = list(df2['Shannon Capacity'])

  latencysC1 = np.array(auxlatencysC1)
  latencysC2 = np.array(auxlatencysC2)

  capacitiesC1 = np.array(df1['Shannon Capacity'])
  capacitiesC2 = np.array(df2['Shannon Capacity'])

  #print("Latency c1: ", latencysC1)
  #print("Latency c2: ", latencysC2)
  #print("Shape: ", df.shape)

  rows1, columns1 = latencysC1.shape
  rows2, columns2 = latencysC2.shape

  bwps1 = np.array(df1['BWP [MHz]'])
  bwps2 = np.array(df2['BWP [MHz]'])

  onL1 = np.array(df1['onTime [ms]'])
  onL2 = np.array(df2['onTime [ms]'])

  cycle1 = np.array(df1['Cycle Length [ms]'])
  cycle2 = np.array(df2['Cycle Length [ms]'])


  for bwp in bwps1:
    if bwp == 25000:
      kmax1.append(5)
    elif bwp == 30000:
      kmax1.append(6)
    elif bwp == 40000:
      kmax1.append(8)
    elif bwp == 50000:
      kmax1.append(10)

  for bwp in bwps2:
    if bwp == 25000:
      kmax2.append(5)
    elif bwp == 30000:
      kmax2.append(6)
    elif bwp == 20000:
      kmax2.append(4)
    elif bwp == 5000:
      kmax2.append(1)
    elif bwp == 10000:
      kmax2.append(2)
    elif bwp == 15000:
      kmax2.append(3)

  auxx = [0]*columns1
  aux1 = [auxx]*rows1
  for column1 in range(columns1):
    auxList = []
    for kmax in kmax1:
      if kmax1.index(kmax) not in auxList:
        indexes = [kmax1.index(kmax)]
        for kmaxx in kmax1[kmax1.index(kmax):]:
          if kmaxx == kmax:
            indexes.append(kmax1.index(kmaxx))

        auxList.extend(indexes)

        capacitiesUEs = []
        for ue in indexes:
          capacitiesUEs.append(capacitiesC1[ue])

        auxOrd = []
        maxx = 0
        for i in range(len(indexes)):
          maxx = max(capacitiesUEs)
          auxOrd.append(capacitiesUEs.index(maxx))
          capacitiesUEs[capacitiesUEs.index(maxx)] = 0

        uesordered = []
        for ueord in auxOrd:
          uesordered.append(indexes[ueord])


        k = 0
        qtdGroup = onL1[indexes[0]]//kmax
        group = 0
        for ue in uesordered:
          if k % kmax == 0 and k != 0:
            group += 1
            if group >= qtdGroup and group % qtdGroup == 0:
              k = 0

          if group < qtdGroup:
            if k <= kmax:
              aux1[ue][column1] = 0
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = 7
            elif k > 8*kmax:
              aux1[ue][column1] = 8

            k += 1

          elif group >= qtdGroup and group < qtdGroup*2:
            if k <= kmax:
              aux1[ue][column1] = cycle1[ue]
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = cycle1[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = cycle1[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = cycle1[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = cycle1[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = cycle1[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = cycle1[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = cycle1[ue] + 7
            elif k > 8*kmax:
              aux1[ue][column1] = cycle1[ue] + 8

            k += 1
          elif group >= qtdGroup*2 and group < qtdGroup*3:
            if k <= kmax:
              aux1[ue][column1] = 2*cycle1[ue]
            elif k > kmax and k <= 2*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 7
            elif k > 8*kmax:
              aux1[ue][column1] = 2*cycle1[ue] + 8

            k += 1

  auxxx = [0]*columns2
  aux2 = [auxxx]*rows2
  for column2 in range(columns2):
    auxList = []
    for kmax in kmax2:
      if kmax2.index(kmax) not in auxList:
        indexes = [kmax2.index(kmax)]
        for kmaxx in kmax2[kmax2.index(kmax):]:
          if kmaxx == kmax:
            indexes.append(kmax2.index(kmaxx))

        auxList.extend

        capacitiesUEs = []
        for ue in indexes:
          capacitiesUEs.append(capacitiesC1[ue])

        auxOrd = []
        maxx = 0
        for i in range(len(indexes)):
          maxx = max(capacitiesUEs)
          auxOrd.append(capacitiesUEs.index(maxx))
          capacitiesUEs[capacitiesUEs.index(maxx)] = 0

        uesordered = []
        for ueord in auxOrd:
          uesordered.append(indexes[ueord])

        k = 0
        qtdGroup = onL2[indexes[0]]//kmax
        group = 0
        for ue in uesordered:
          if k % kmax == 0 and k != 0:
            group += 1
            if group >= qtdGroup and group % qtdGroup == 0:
              k = 0

          if group < qtdGroup:
            if k <= kmax:
              aux2[ue][column2] = 0
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = 7
            elif k > 8*kmax:
              aux2[ue][column2] = 8

            k += 1

          elif group >= qtdGroup and group < qtdGroup*2:
            if k <= kmax:
              aux2[ue][column2] = cycle2[ue]
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = cycle2[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = cycle2[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = cycle2[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = cycle2[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = cycle2[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = cycle2[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = cycle2[ue] + 7
            elif k > 8*kmax:
              aux2[ue][column2] = cycle2[ue] + 8

            k += 1
          elif group >= qtdGroup*2 and group < qtdGroup*3:
            if k <= kmax:
              aux2[ue][column2] = 2*cycle2[ue]
            elif k > kmax and k <= 2*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 1
            elif k > 2 *kmax and k <= 3*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 2
            elif k > 3*kmax and k <= 4*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 3
            elif k > 4*kmax and k <= 5*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 4
            elif k > 5*kmax and k <= 6*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 5
            elif k > 6*kmax and k <= 7*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 6
            elif k > 7*kmax and k <= 8*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 7
            elif k > 8*kmax:
              aux2[ue][column2] = 2*cycle2[ue] + 8

            k += 1

  df1['DCI_time'] = aux1
  df2['DCI_time'] = aux2

  df = pd.concat([df1, df2], ignore_index=True)
  #print(df)
  return df

In [ ]:
def transmissionTime(df, tcs):
  sizes = np.array(df['Packet Size [bytes]'])
  capacities = np.array(df['Shannon Capacity'])
  tT = []

  for size, capacity in zip(sizes, capacities):
    aux = []
    for tc in tcs:
      aux.append(size/capacity)  #196.08 bytes/ms = 196608 bytes/s = 1.5mbs
    tT.append(aux)

  df['Transmission_time'] = tT

  return df

In [ ]:
def totalLatency(df):
  latencyaux = list(df['Latency_Cycle'])
  tTaux = list(df['Transmission_time'])
  dciaux = list(df['DCI_time'])

  latencyCycle = np.array(latencyaux)
  dci = np.array(dciaux)
  tT = np.array(tTaux)

  totalLatencyaux = latencyCycle + dci
  totalLatency = totalLatencyaux + tT
  df['Total_Latency'] = totalLatency.tolist()
  #print(df)
  return df

In [ ]:
def lista_simples(lista):
  if isinstance(lista, list):
      return [sub_elem for elem in lista for sub_elem in lista_simples(elem)]
  else:
      return [lista]

In [ ]:
def objectiveFunction(x, nURLLC, nEMBB):
  # x[0] = variável para otimizar a latência em função do omega
  # x[1] = variável para calcular a quantidade de ciclo ON

  #qtdCy = 400
  #tcx = int(x[0])
  #tc = [tcx for tcx in range (tcx, qtdCy, tcx)]
  tc = arrivalTimePoisson()

  UEs.clear()
  UEsAux.clear()
  if nURLLC > nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nEMBB:
        adiciona(2, x)
      else:
        adiciona(1, x)
  elif nURLLC < nEMBB:
    for i in range(0, (nURLLC + nEMBB)):
      if i < nURLLC:
        adiciona(1, x)
      else:
        adiciona(2, x)

  df = inicializaUE()
  df = scheduler(df)
  df = shannonCapacity(df)
  df = latencyCycle(df, tc)
  df = DCI(df)
  df = transmissionTime(df, tc)
  df = totalLatency(df)
  #print(df)
  latency = (np.array(df['Total_Latency']))
  auxlist = []
  auxlist.append(latency.tolist())
  Lit.extend(lista_simples(auxlist))
  power(df)


  sum2 = 0


  for i in range(0, len(UEs)):
      sum1 = 0
      sum2 = 0
      for j in range(0, len(T)):
          aux = (beta[i] * Pit[i])/ Pmax
          sum1 += aux

      sum2 += sum1
  return sum2


#restricoes
def restricao(x):
  sum1 = sum(x[0:23])
  sum2 = sum(x[23:47])
  sum3 = sum(x[47:51])
  sum4 = sum(x[51:53])
  sum5 = sum(x[53:73])
  sum6 = sum(x[73:123])
  sum7 = sum(x[123:129])
  sum8 = sum(x[129:131])
  #sum9 = sum(x[131])
  return (sum1 + sum2 + sum3 + sum4 + sum5 + sum6 + sum7 + sum8 - 8)

cons = [{'type': 'ineq', 'fun': restricao}]


# limites
#a = [10, 20, 50, 100, 200]
#a = [5, 10, 20, 50, 100]
#b = [1000, 100000]  # Valores possíveis de wij

a = tuple(131*[[0, 1]])


bnds = a



In [ ]:
from time import time

In [ ]:
# estimativa inicial
x0 = 131*[1]

#Vetores para guardar a latência em cada caso
latency_20_80_3 = []
latency_40_60_3 = []
latency_60_40_3 = []
latency_80_20_3 = []

#Vetores para guardar a quantidade de usuários na rede

#Vetores para guardar a potência em cada caso
power_20_80_3 = []
power_40_60_3 = []
power_60_40_3 = []
power_80_20_3 = []

NURLLC = 20
NeMBB = 80

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_20_80_3.extend(Lit)
  power_20_80_3.extend(Pit)

Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 40
NeMBB = 60

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_40_60_3.extend(Lit)
  power_40_60_3.extend(Pit)


Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 60
NeMBB = 40

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_60_40_3.extend(Lit)
  power_60_40_3.extend(Pit)


Lit.clear()
Pit.clear()
#UEs.clear()

In [ ]:
NURLLC = 80
NeMBB = 20

tempo = 4
start = time()
while (time() - start) <= tempo:

  solution = minimize(objectiveFunction, x0, method='SLSQP', bounds=bnds, constraints=cons, args=(NURLLC, NeMBB))
  latency_80_20_3.extend(Lit)
  power_80_20_3.extend(Pit)


Lit.clear()
Pit.clear()
UEs.clear()

In [ ]:
sortL_3 = np.sort(latency_20_80_3)
sortL2_3 = np.sort(latency_40_60_3)
sortL3_3 =  np.sort(latency_60_40_3)
sortL4_3 = np.sort(latency_80_20_3)


pL_3 = 1.0 * np.arange(len(sortL_3)) / float(len(sortL_3) - 1)
pL2_3 = 1.0 * np.arange(len(sortL2_3)) / float(len(sortL2_3) - 1)
pL3_3 = 1.0 * np.arange(len(sortL3_3)) / float(len(sortL3_3) - 1)
pL4_3 = 1.0 * np.arange(len(sortL4_3)) / float(len(sortL4_3) - 1)

plt.plot(sortL_3, pL_3, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_3, pL2_3, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_3, pL3_3, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_3, pL4_3, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
sortP_3 = np.sort(power_20_80_3)
sortP2_3 = np.sort(power_40_60_3)
sortP3_3 =  np.sort(power_60_40_3)
sortP4_3 = np.sort(power_80_20_3)

pP_3 = 1.0 * np.arange(len(sortP_3)) / float(len(sortP_3) - 1)
pP2_3 = 1.0 * np.arange(len(sortP2_3)) / float(len(sortP2_3) - 1)
pP3_3 = 1.0 * np.arange(len(sortP3_3)) / float(len(sortP3_3) - 1)
pP4_3 = 1.0 * np.arange(len(sortP4_3)) / float(len(sortP4_3) - 1)

plt.plot(sortP_3, pP_3, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_3, pP2_3, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_3, pP3_3, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_3, pP4_3, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.show()

In [ ]:
plt.subplot(4, 1, 1)
plt.plot(sortL_1, pL_1, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_1, pL2_1, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_1, pL3_1, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_1, pL4_1, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Shannon Capacity')

plt.subplot(4, 1, 2)
plt.plot(sortL_2, pL_2, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_2, pL2_2, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_2, pL3_2, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_2, pL4_2, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Algorithm 1')

plt.subplot(4, 1, 3)
plt.plot(sortL_3, pL_3, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortL2_3, pL2_3, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortL3_3, pL3_3, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortL4_3, pL4_3, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('latency [ms]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Algorithm 2')

plt.show()

In [ ]:
plt.subplot(4, 1, 1)
plt.plot(sortP_1, pP_1, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_1, pP2_1, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_1, pP3_1, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_1, pP4_1, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Shannon Capacity')

plt.subplot(4, 1, 2)
plt.plot(sortP_2, pP_2, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_2, pP2_2, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_2, pP3_2, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_2, pP4_2, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Algorithm 1')

plt.subplot(4,1,3)
plt.plot(sortP_3, pP_3, "-g", label="20 - URLLC; 80 - eMBB")
plt.plot(sortP2_3, pP2_3, "-r", label="40 - URLLC; 60 - eMBB")
plt.plot(sortP3_3, pP3_3, "-b", label="60 - URLLC; 40 - eMBB")
plt.plot(sortP4_3, pP4_3, "-y", label="80 - URLLC; 20 - eMBB")
plt.legend(loc="lower right", fontsize=14)
plt.xlabel('Power [mW]', fontsize=16)
plt.ylabel('CDF', fontsize=16)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.title('Algorithm 2')

plt.show()